In [ ]:
import cv2
print(cv2.__version__)

In [1]:
import cv2
#print(cv2.__version__)

img = cv2.imread("C:\img_source\img\himawari.png")
#print(type(img))
#print(img)

cv2.imshow('original', img)
cv2.waitKey()
cv2.destroyAllWindows()

#グレスケに変換
imgGray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

cv2.imshow('Gray Image', imgGray)
cv2.waitKey()
cv2.destroyAllWindows()

################
## ガンマ補正 ##
################

#ガンマ値
gamma = 0.1

# 画素値の最大値
imax = imgGray.max()

# ガンマ補正
imgGamma = imax * (imgGray / imax)**(1/gamma)

cv2.imshow('Gamma Image', imgGamma)
cv2.waitKey()
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np

#面積が最大の長方形のインデックスを返す関数
def detect_max_rect(img,contours):
    #最大面積をとるcontoursのインデックス
    max_idx = 0
    
    for i in range(len(contours)):
        if cv2.contourArea(contours[max_idx]) < cv2.contourArea(contours[i],
        window_menseki*0.99 > cv2.contourArea(contours[max_idx])                                                      ):
            max_idx = i
    
    return max_idx


# VideoCapture オブジェクトを取得します
capture = cv2.VideoCapture(0)
window_menseki = 800*600

while(True):
    ret, frame = capture.read()
    # resize the window
    windowsize = (800, 600)
    frame = cv2.resize(frame, windowsize)
    
    imgray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    imblur = cv2.GaussianBlur(imgray, (11, 11), 0)
    ret,thresh = cv2.threshold(imblur, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    im, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    target_contour = contours[detect_max_rect(frame,contours)]

    rect = cv2.minAreaRect(target_contour)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(frame, [box], 0, (0,255,0), 2)
    
    cv2.imshow('title',thresh)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np

#切り出し関数
def crop_rect(img, rect):
    center, size, angle = rect
    center = tuple(map(int, center))  # float -> int
    size = tuple(map(int, size))  # float -> int
    img_w, img_h = img.shape[:2]  # 画像の高さ、幅
    
    #cv2.circle(img,center,5,(0,0,255))

    #rectの高さと幅を取得
    height,width=size
    height,width=int(height),int(width)
    
    print(angle)
    
    #angleが水平方向の線とのなす角を示すので，angle<-45°の時は＋９０してwidthとheightを入れ替えることで1,2象限までは上向きにできる
    if angle<-45:
        angle+=90
        height,width = width,height
            
    #affine行列を用いて元画像を回転させる
    M = cv2.getRotationMatrix2D(center, angle, 1)
    rotated = cv2.warpAffine(img, M, (img_h,img_w))
    #cv2.imshow('rotated', rotated)

    # 切り抜く
    cropped = cv2.getRectSubPix(rotated, (height,width), center)
    
    # ---------- For Debug ------------#
    
    #緑の四角で囲む
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(img, [box], 0, (0,255,0), 2)
    
    cv2.imshow('add box', img)
    
    # ---------- For Debug ------------#

    return cropped

#四角形を抽出する関数
def get_image(img):
    # cannyで輪郭を抽出する
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (9, 9), 0)
    canny = cv2.Canny(imgBlur, 50, 100)
    
    #モルフォロジー変換の膨張(Dilation)をする
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    dilate = cv2.dilate(canny, kernel, iterations=1)
    
    #二値化する
    #thresh = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #thresh = cv2.GaussianBlur(thresh, (5, 5), 0)
    #ret,thresh = cv2.threshold(thresh,115,255,0)
    #cv2.imshow('thresh', thresh)
    
    #輪郭を抽出して，面積の大きい順にソート
    cnts = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)[1]#輪郭抽出  
    cnts.sort(key=lambda x:cv2.contourArea(x), reverse=True)  # 面積が大きい順にソート
    
    #矩形を斜めに抽出したいのでminAreaRect()を使用
    rect = cv2.minAreaRect(cnts[0])
    
    #最大面積部分を切り取る
    croped_img = crop_rect(img,rect)
    
    # ---------- For Debug ------------#
    #上位3つを大きい順に赤のグラデーションで
    out = np.zeros_like(img)
    cv2.drawContours(out, [cnts[0]], -1, color=(0,0,255), thickness=-1)
    #cv2.drawContours(out, [cnts[1]], -1, color=(0,0,128), thickness=-1)
    #cv2.drawContours(out, [cnts[2]], -1, color=(0,0,64), thickness=-1)
        
    #print(cnts[0])
    cv2.imshow('croped_img', croped_img)
    #cv2.imshow('max_red', out)
    cv2.imshow('canny', dilate)
    # ---------- For Debug ------------#
    
    return croped_img
    
    
# VideoCapture オブジェクトを取得します
capture = cv2.VideoCapture(0)
#windowサイズ
windowsize = (800, 600)
    
#始めに置き換えたい四角形を抽出するためのboolean
convert_img_flag = False
    

while(True):
    ret,frame = capture.read()
    frame = cv2.resize(frame, windowsize)
    
    
    cv2.imshow('title',frame)
    
    
    
    key = cv2.waitKey(1) & 0xFF
    #"Q"を押したらプログラム修了
    if key == ord('q'):
        break
    #"E"を押したら画面上の一番大きい四角形を抽出
    if key == ord('e') and convert_img_flag == False:
        advertise_img = get_image(frame)
        convert_img_flag = True
    #"A"を押したらもう一度上の操作が可能になる
    if key == ord('d'):
        convert_img_flag = False
        
capture.release()
cv2.destroyAllWindows()

-0.0


In [10]:
#写真のリサイズ

from PIL import Image

img = Image.open("../img/doradora2.jpg")
def image_resize(width, image_name):
    resize_image = img.resize((width, int(width * img.size[1] / img.size[0])))  # 画像のリサイズ
    resize_image.save('../img/' + image_name)  # 画像の保存

image_resize(500, 'doradora1.jpg')

In [5]:
#検出された四角形全てに画像をペーストするコード
import cv2
import numpy as np
from PIL import Image
import time

#四角形を抽出する関数
def get_image(img):
    ######################################
    # 矩形を正確に抽出するための加工部分 #
    ######################################
    
    #小さい領域を無視するための閾値
    th_area = img.shape[0] * img.shape[1] / 100
    
    # cannyで輪郭を抽出する
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (9, 9), 0)
    canny = cv2.Canny(imgBlur, 50, 100)
    
    #モルフォロジー変換の膨張(Dilation)をする
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    dilate = cv2.dilate(canny, kernel, iterations=1)
    
    #二値化する
    #thresh = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #thresh = cv2.GaussianBlur(thresh, (5, 5), 0)
    #ret,thresh = cv2.threshold(thresh,115,255,0)
    #cv2.imshow('thresh', thresh)
    
    #コーナー座標抽出
    contours = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[1] 
    contours.sort(key=lambda x:cv2.contourArea(x), reverse=True)  # 面積が大きい順にソート
    
    #閾値以上の面積を持つものだけにする
    #大きすぎるものも除く(今回は0.8倍未満のもの))
    #四角形だけに絞る
    areas = []
    for cnt in contours:
        if th_area < cv2.contourArea(cnt) < img.shape[0] * img.shape[1]*0.80:          
            epsilon = 0.1*cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,epsilon,True)
            if len(approx) == 4:
                areas.append(approx)    
    
    ##############################################
    # 矩形を正確に抽出するための加工部分 ここまで#
    ##############################################
    
    #例外処理
    #一番大きいやつ
    if contours == 0:
        max_contour = [(0,0,0,0)]
    else:
        max_contour = contours[0]
    if len(areas) == 0:
        flag=False
        return 0,0,flag
    
    #射影変換
    dst = []

    pts1 = np.float32(areas[0])
    pts2 = np.float32([[500,0],[0,0],[0,500],[500,500]])

    M = cv2.getPerspectiveTransform(pts1,pts2)
    dst = cv2.warpPerspective(img,M,(500,500))    
    # ---------- For Debug -----------------------------------------#
        
    #検出された輪郭が4つだけの四角形を赤色で描画
    #cv2.drawContours(img,contours[0],-1,(0,0,255), thickness=5)   
    #cv2.imshow('advertise_img', img)
    #print(areas[0])
    #cv2.imshow('dst', dst)
    #cv2.imshow('canny', canny)
    #cv2.imshow('dilate', dilate)    
    # ---------- For Debug ------------------------------------------#
    
    flag=True
    return dst,areas,flag

#imgにad_imgを合成する関数
def convert_img(img,ad_img):
    
    advertise_img,areas,flag = get_image(frame)

    #一個もない時の例外処理
    if areas == 0:
        overlayed_img = frame
        cv2.imshow('overlaid_img', overlayed_img)
        return 0
        #areas = []
        #areas.append(np.float32([[0,0],[0,0],[0,0],[0,0]]))
    
    pts1 = np.float32([[500,0],[0,0],[0,500],[500,500]])
    pts2 = [np.float32(areas[i]) for i in range(len(areas)) ]
    
    #ad_imgをimgの座標に合うようにホモグラフィ行列を作成
    for i in range(len(pts2)):
        
        M = cv2.getPerspectiveTransform(pts1,pts2[i])
    
        #移動させる．引数のborderValueで背景が黒になるように設定（デフォルトでも0）
        dst = cv2.warpPerspective(ad_img,M,(img.shape[1],img.shape[0]), flags = cv2.INTER_CUBIC
                              ,borderMode=cv2.BORDER_CONSTANT,borderValue=[0,0,0])
    
        #print(dst.shape)
        trans_img = convert_to_trans(dst,img)
        
        #一回目のみimgに直接オーバーライド
        if i == 0:
            overlayed_img = overlay_img(img,trans_img)
        else:
            overlayed_img = overlay_img(overlayed_img,trans_img)
            
    #cv2.imshow('img', img)
    #cv2.imshow('dst', dst)
    #cv2.imshow('trans_img', trans_img)
    cv2.imshow('overlaid_img', overlayed_img)
    #cv2.imwrite('../img/output/output.jpg',dst) 
    
#黒を透明にする関数
def convert_to_trans(src,dst):
    #dst=cv2.cvtColor(dst, cv2.COLOR_RGB2RGBA)
    h,w=src.shape[:2]
    
    hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
    
    lower = np.array([0, 0, 0])
    upper = np.array([0, 0, 0])
    
    masking_color = cv2.inRange(hsv, lower,upper)

    mask = cv2.bitwise_not(src, src, mask=masking_color)
    mask = cv2.bitwise_not(masking_color)

    bgr = cv2.split(src)

    img_alpha = cv2.merge(bgr+[mask])
    
    #dst[:,:]=img_alpha

    #cv2.imshow("test", img_alpha)
    #cv2.imwrite('../img/output/out.png', img_alpha)

    return img_alpha
    
#imgに透過画像をオーバーライドさせる関数    
def overlay_img(img,trans_img):
    h,w=img.shape[:2]
    
    #cv->pillowなので，BGR->RGB
    new_img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA)
    pil_rgb_img = Image.fromarray(new_img)
    pil_rgba_img = pil_rgb_img.convert('RGBA')

    new_trans_img = cv2.cvtColor(trans_img, cv2.COLOR_BGRA2RGBA)
    pil_rgb_trans_img = Image.fromarray(new_trans_img)
    pil_rgba_trans_img = pil_rgb_trans_img.convert('RGBA')

    #空の背景画像を作成
    pil_rgba_temp = Image.new('RGBA', pil_rgba_img.size,(255, 255, 255, 0))
    
    # 座標を指定し重ね合わせる
    pil_rgba_temp.paste(pil_rgba_trans_img, (0,0), pil_rgba_trans_img)
    result_img = Image.alpha_composite(pil_rgba_img, pil_rgba_temp)

    #pillow->cvなので，RGB->BGR
    cv_return_img = cv2.cvtColor(np.array(result_img), cv2.COLOR_RGBA2BGRA)

    #cv2.imshow('title',cv_return_img)
    
    return cv_return_img
    
#四角領域を赤枠で囲む関数
def draw_corner(img):
    
    ######################################
    # 矩形を正確に抽出するための加工部分 #
    ######################################
    
    #小さい領域を無視するための閾値
    th_area = img.shape[0] * img.shape[1] / 100
    
    # cannyで輪郭を抽出する
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (9, 9), 0)
    canny = cv2.Canny(imgBlur, 50, 100)
    
    #モルフォロジー変換の膨張(Dilation)をする
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    dilate = cv2.dilate(canny, kernel, iterations=1)
    
    #二値化する
    #thresh = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #thresh = cv2.GaussianBlur(thresh, (5, 5), 0)
    #ret,thresh = cv2.threshold(thresh,115,255,0)
    #cv2.imshow('thresh', thresh)
    
    #コーナー座標抽出
    contours = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[1] 
    contours.sort(key=lambda x:cv2.contourArea(x), reverse=True)  # 面積が大きい順にソート
    
    #閾値以上の面積を持つものだけにする
    #大きすぎるものも除く(今回は0.8倍未満のもの))
    #四角形だけに絞る
    areas = []
    for cnt in contours:
        if th_area < cv2.contourArea(cnt) < img.shape[0] * img.shape[1]*0.80:          
            epsilon = 0.1*cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,epsilon,True)
            if len(approx) == 4:
                areas.append(approx)
                  
    ##############################################
    # 矩形を正確に抽出するための加工部分 ここまで#
    ##############################################
    
    #一番大きいやつ
    max_contour = contours[0]
    

    #例外処理
    if len(areas) == 0:
        return 0
    
    cv2.drawContours(img,max_contour,-1,(0,0,255), thickness=5) 
    
    return img
    
###ここからグローバル変数＋main文

image_dir = '../img/'
image_file = 'doradora1.png'

#始めに置き換えたい四角形を抽出するためのboolean
convert_img_flag = False

# VideoCapture オブジェクトを取得します
capture = cv2.VideoCapture(0)
#windowサイズ
windowsize = (800, 600)

while(True):
    #img = cv2.imread(image_dir+image_file)
    frame = capture.read()[1]
    #frame = cv2.resize(img, windowsize)
    #frame=img
    
    #frame = draw_corner(frame)
    
    cv2.imshow('title',frame)

    if convert_img_flag == True:
        convert_img(frame,advertise_img)
    
    key = cv2.waitKey(1) & 0xFF
    #"Q"を押したらプログラム修了
    if key == ord('q'):
        break
    #"E"を押したら画面上の一番大きい四角形を抽出
    if key == ord('e') and convert_img_flag == False:
        advertise_img,areas,flag = get_image(frame)
        if flag==False:
            break
        else:
            convert_img_flag = True
    #"A"を押したらもう一度上の操作が可能になる
    if key == ord('d'):
        convert_img_flag = False
capture.release()
cv2.destroyAllWindows()
